<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2023notebooks/2023_1101Sentence_generation_demo_with_JapaneseStableLM_3B_4E1TBase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM による単文作成デモ

- source: https://huggingface.co/stabilityai/japanese-stablelm-instruct-gamma-7b
- date: 2023_1101
- author: 浅川伸一
- filename: `2023_1101Sentence_generation_demo_with_JapaneseStableLM-3B-4E1TBase.ipynb`

In [ ]:
# Transformers >= 4.34.0 でないと動作しないようなのでアップデート
# というか Colab にはそもそも transformers がインストールされていない
import IPython
isColab = 'google.colab' in str(IPython.get_ipython())

if isColab:
    !pip install --upgrade transformers

In [2]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("stabilityai/japanese-stablelm-3b-4e1t-base")
model = AutoModelForCausalLM.from_pretrained(
  "stabilityai/japanese-stablelm-3b-4e1t-base",
  trust_remote_code=True,
  torch_dtype="auto",
)
model = model.to(device)
model.eval()

In [4]:
# 下記の gamma-7b モデルではメモリ不足でクラッシュしてしまうので
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("stabilityai/japanese-stablelm-instruct-gamma-7b")
# model = AutoModelForCausalLM.from_pretrained(
#   "stabilityai/japanese-stablelm-instruct-gamma-7b",
#   torch_dtype="auto",
# )
# model = model.to(device)
# model.eval()

In [4]:
def build_prompt(user_query, inputs="", sep="\n\n### "):
    sys_msg = "以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。"
    p = sys_msg
    roles = ["指示", "応答"]
    msgs = [": \n" + user_query, ": \n"]
    if inputs:
        roles.insert(1, "入力")
        msgs.insert(1, ": \n" + inputs)
    for role, msg in zip(roles, msgs):
        p += sep + role + msg
    return p

In [5]:
# Infer with prompt without any additional input
user_inputs = {
    "user_query": "与えられたことわざの意味を小学生でも分かるように教えてください。",
    "inputs": "情けは人のためならず"
}
prompt = build_prompt(**user_inputs)

input_ids = tokenizer.encode(
    prompt,
    add_special_tokens=False,
    return_tensors="pt"
)

tokens = model.generate(
    input_ids.to(device=model.device),
    max_new_tokens=256,
    temperature=1,
    top_p=0.95,
    do_sample=True,
)

out = tokenizer.decode(tokens[0][input_ids.shape[1]:], skip_special_tokens=True).strip()
print(out)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


「情けは人のためならず」ということわざは、実際には、よく似た意味をもつ別のことわざと、同一視されている。前者は「情けは巡り巡って己に帰り来る」と、後者は「情けは人に倍する」とされている。どちらも正しいが、どちらも、単に情けは巡り巡って自分に帰ってくるという、原文の通りの解釈にはならない。これは、文脈からの判断を行い、言葉を使い分ける必要があるからである。

さて、「情けは人のためならず」ということわざについて考えてみよう。私たちは普段、自分の行動を「正しいか」とか「悪いか」とかを考えるが、「正しい」「悪い」の判断を


In [30]:
user_inputs = {
    "user_query": "次の文と同じような例文を考えてください。",
    #"user_query": "与えられた文の難易度と同程度の文を生成してください。",
    "inputs": "彼女は音楽が好きなようで、毎日、ＣＤを聞いたり歌を歌ったりピアノをひいたりしている。" # ソースははごろも
}
prompt = build_prompt(**user_inputs)

input_ids = tokenizer.encode(
    prompt,
    add_special_tokens=False,
    return_tensors="pt"
)

tokens = model.generate(
    input_ids.to(device=model.device),
    max_new_tokens=256,
    #temperature=0.5,
    temperature=1,
    top_p=0.95,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

out = tokenizer.decode(tokens[0][input_ids.shape[1]:], skip_special_tokens=True).strip()
print(f'--- 得られた出力:---\n{out}')


--- 得られた出力:---
彼女はピアノ演奏に向いているようで、音大を卒業してから、教育機関に勤務しながら教鞭を執っている。


In [ ]:
# こちらは松尾研のバージョンなので無視してください。
# source: https://huggingface.co/matsuo-lab/weblab-10b-instruction-sft
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("matsuo-lab/weblab-10b-instruction-sft")
model = AutoModelForCausalLM.from_pretrained("matsuo-lab/weblab-10b-instruction-sft", torch_dtype=torch.float16)

if torch.cuda.is_available():
    model = model.to("cuda")

text = "大規模言語モデルについて説明してください。"
text = f'以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n{text}\n\n### 応答:'
token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7,
        top_p=0.95
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)
